Импортируем библиотеки

In [1]:
import requests as req
import bs4  as bs
import pandas as pd
import sqlite3

Определяем константы и перемнные

In [2]:
URL_SEARCH = "https://hh.ru/search/vacancy"
URL_HEADERS = {'User-Agent': 'Mozilla/5.0'}
DB_NAME = 'pars.db'
TABLE_NAME = 'vacancies'
LIMIT_OF_VACANCY_DETAIL = 100

In [3]:
numersOfPage = 0
listLinkOfVacancy = []
vacancyDetailInfo =[]
url_params = {'text':'python middle',
              'per_page':25,
              'page':0}

Определяем функции

In [4]:
# Загружаем страницу и возвращаем BeautifulSoup
def getBSPage(url,header,param):
    result = req.get(url, headers=header, params=param)
    return bs.BeautifulSoup(result.content,'lxml')

In [5]:
# Функция для парсинга страницы и записи ссылок на вакансии
def addVacancyToList(soup):
    for vac in list(soup.find_all('a',{'data-qa': 'serp-item__title','target':'_blank'})):
        vac = str(vac)
        if vac.find('hh.ru/vacancy/') > 0 :
            listLinkOfVacancy.append('https://' + vac[vac.find('hh.ru/vacancy/'):vac.find('?')])
    print(f'Список ссылок вакансий содержит {len(listLinkOfVacancy)} записей')

In [6]:
# Функция для получения названия копании
def getCompanyName(soup):
    companyName = soup.find('a',{'data-qa':'vacancy-company-name','class':'bloko-link bloko-link_kind-tertiary'})
    if (companyName is not None):
        return companyName.text

In [7]:
# Функция для получения названия вакансии
def getVacancyName(soup):
    vacancyName = soup.find('h1', {'class':'bloko-header-section-1','data-qa':'vacancy-title'})
    if (vacancyName is not None):
        return vacancyName.text

In [8]:
# Функция для получения описания вакансии
def getVacancyDescription(soup):
    vacancyDecription = ''
    for row in soup.find_all('div',{'class':'vacancy-section'}):
        if (row.find('div',{'class':'g-user-content','data-qa':'vacancy-description'}) is not None):
            vacancyDecription += str(row)
    return vacancyDecription

In [9]:
# Функция для получения списка ключевых навыков
def getVacancyRequirements(soup):
    listOfRequirements = []
    vacancies = soup.find_all('div', {'class': 'vacancy-section'})
    for vacancy in vacancies:
        if vacancy.find('h2',{'class':'bloko-header-section-2'}):
            requirements = vacancy.find_all('span')
            for requirement in requirements:
                listOfRequirements.append(requirement.text)
    stringOfRequirements = ', '.join(listOfRequirements)
    return stringOfRequirements

Собираем информацию о всех вакансиях

In [10]:
# Получаем кол-во страниц с результатами поиска, собираем список вакансий
soup = getBSPage(URL_SEARCH,URL_HEADERS,url_params)
numers_of_page = int(soup.find_all('a',{'data-qa': 'pager-page'})[-1].text)
for i in range(numers_of_page + 1):
    if i == 0:
        addVacancyToList(soup)
    else:
        url_params['page'] = i
        soup = getBSPage(URL_SEARCH,URL_HEADERS,url_params)
        addVacancyToList(soup)

Список ссылок вакансий содержит 18 записей
Список ссылок вакансий содержит 38 записей
Список ссылок вакансий содержит 58 записей
Список ссылок вакансий содержит 78 записей
Список ссылок вакансий содержит 98 записей
Список ссылок вакансий содержит 118 записей
Список ссылок вакансий содержит 138 записей
Список ссылок вакансий содержит 158 записей
Список ссылок вакансий содержит 178 записей
Список ссылок вакансий содержит 198 записей
Список ссылок вакансий содержит 218 записей
Список ссылок вакансий содержит 238 записей
Список ссылок вакансий содержит 258 записей
Список ссылок вакансий содержит 278 записей
Список ссылок вакансий содержит 298 записей
Список ссылок вакансий содержит 318 записей
Список ссылок вакансий содержит 338 записей
Список ссылок вакансий содержит 358 записей
Список ссылок вакансий содержит 358 записей


Собираем информацию по всем вакансиям из созданного ранее списка

In [11]:

for link in listLinkOfVacancy:
    vacancyRow = []
    soup = getBSPage(link,URL_HEADERS,url_params)
    companyName = getCompanyName(soup)
    vacancyName = getVacancyName(soup)
    vacancyDescription = getVacancyDescription(soup)
    vacancyRequirements = getVacancyRequirements(soup)
    vacancyRow.append(companyName)
    vacancyRow.append(vacancyName)
    vacancyRow.append(vacancyDescription)
    vacancyRow.append(vacancyRequirements)
    vacancyDetailInfo.append(vacancyRow)
    print(f'Список детальной информации о вакансиях содержит {len(vacancyDetailInfo)} записей')
    # Этим куском можно регулировать количество интересующих данных
    if (len(vacancyDetailInfo) >= LIMIT_OF_VACANCY_DETAIL):
        break

Список детальной информации о вакансиях содержит 1 записей
Список детальной информации о вакансиях содержит 2 записей
Список детальной информации о вакансиях содержит 3 записей
Список детальной информации о вакансиях содержит 4 записей
Список детальной информации о вакансиях содержит 5 записей
Список детальной информации о вакансиях содержит 6 записей
Список детальной информации о вакансиях содержит 7 записей
Список детальной информации о вакансиях содержит 8 записей
Список детальной информации о вакансиях содержит 9 записей
Список детальной информации о вакансиях содержит 10 записей
Список детальной информации о вакансиях содержит 11 записей
Список детальной информации о вакансиях содержит 12 записей
Список детальной информации о вакансиях содержит 13 записей
Список детальной информации о вакансиях содержит 14 записей
Список детальной информации о вакансиях содержит 15 записей
Список детальной информации о вакансиях содержит 16 записей
Список детальной информации о вакансиях содержит 

Записываем результат в БД

In [12]:
columns = ['company_name', 'position', 'job_description', 'key_skills']
df = pd.DataFrame(data=vacancyDetailInfo,columns=columns)

In [13]:
connection = sqlite3.connect(DB_NAME)
df.to_sql(TABLE_NAME,connection,if_exists='replace',index=False)
connection.close()